In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
#loading data
train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')

In [ ]:
train.head() #returns first five rows

In [ ]:
train.columns #returns names of and columns

In [ ]:
#removing unwanted columns
train.drop(['id'],axis = 1, inplace = True)

In [ ]:
train.head()

In [ ]:
#checking null values
train.isnull().sum()

In [ ]:
x = train.drop(['loss'], axis=1)
y = train['loss']

In [ ]:
train.shape

In [ ]:
from sklearn.model_selection import train_test_split

# splitting data
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.15,random_state=42)

print("Shape of train set:", X_train.shape)
print("Shape of test set:", X_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

In [ ]:
#importing model
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error


xgb_params = {'n_estimators':5000,'learning_rate': 0.01,'n_jobs':4, 'tree_method':'gpu_hist',
              'max_depth': 3,'min_child_weight': 5, 'random_state':42}
model = XGBRegressor(**xgb_params)
model.fit(X_train, y_train,
          early_stopping_rounds=100, 
          eval_set=[(X_train,y_train),(X_test, y_test)],
          eval_metric = "rmse",
          verbose=False)
preds_test = model.predict(X_test)
print("RMSE:" ,mean_squared_error(y_test, preds_test, squared=False))

In [ ]:
test_df = test.drop(['id'], axis=1)
test_df = ss.fit_transform(test_df)

In [ ]:
#making predictions with test data
pred = model.predict(test_df)

In [ ]:
#submission
df_sub = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')
df_sub['loss'] = pred
df_sub.to_csv('submission19.csv' , index = False)
df_sub.head(10)